In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error,  accuracy_score, precision_score, recall_score, f1_score

def groubpy_func_class_metrics(df, pred_column):
    count = df.shape[0]
    dist = {}
    #print(df)

    out = pd.DataFrame(index = [0])

    out['count'] = count

    for i in range(1, 6):
        out['dist_' + str(i)] = df[df['label'] == i].shape[0]
    
    accuracy_metric = accuracy_score(df['label'], df[pred_column])

    out['accuracy'] = accuracy_metric

    out['precision_micro'] = precision_score(df['label'], df[pred_column], average='weighted', zero_division = np.nan)
    out['recall_micro'] = recall_score(df['label'], df[pred_column], average='weighted', zero_division = np.nan)
    out['f1_score_micro'] = f1_score(df['label'], df[pred_column],  average='weighted', zero_division = np.nan)



    #print(out)
    return out

def id_to_name(id, df_informal_names):
    id_tmp = int(id)
    if id == -1:
        return -1
    else:
        try:
            return df_informal_names.loc[id,:]['EMITENT_FULL_NAME']
        except:
            return id

In [3]:
issuers = pd.read_csv('../../../data/issuers - issuers.csv', index_col = 1).drop('ID человека', axis= 1)

In [4]:
issuers

,EMITENT_FULL_NAME,VeryOddCompany,BGTicker,BGTicker.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
issuerid,,,,,,,,,,,,,,
1,"Акционерный коммерческий банк ""Держава"" публич...",NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",NaN,CBOM RX,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Российский акционерный коммерческий дорожный ...",NaN,NaN,RDRB,Российский акционерный коммерческий дорожный банк,РДБанк,Дорожный банк,Russian public joint-stock commercial roads Bank,RosDorBank,РосДорБанк,roads Bank,NaN,NaN,NaN
4,"Акционерная компания ""АЛРОСА"" (публичное акцио...",NaN,ALRS RX,NaN,алроса,alrosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Акционерный Коммерческий банк ""АВАНГАРД"" - пуб...",NaN,NaN,AVAN,Авангард,AVANGARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,"Публичное акционерное общество ""Новороссийский...",NaN,NaN,NMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,"Публичное акционерное общество ""ВУШ Холдинг""\n",NaN,NaN,WUSH,Whoosh,Вуш,Вхуш,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,"Публичное акционерное общество ""СПБ Биржа""\n",NaN,NaN,SPBE\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
with open('../../../data/messages_many_2/messages_many.pkl', 'rb') as file:

    mes_many = pickle.load(file)

In [6]:
mentions = pd.read_csv('../../../data/sentiment_data_3/mentions.csv', index_col = 0)
mentions = mentions[mentions['issuerid'] > 0]
mentions = mentions.rename(columns={'messageid': 'MessageID'})

In [7]:
sentiment = pd.read_csv('../../../data/sentiment_data_3/sentiment.csv', index_col = 0)
#sentiment = sentiment[sentiment['issuerid'] > 0]
sentiment = sentiment[sentiment['SentimentScore'] != 0]

In [8]:
tmp = sentiment.copy()
tmp['issuerid'] = sentiment['issuerid'].astype('str')
tmp = tmp.groupby(['ChannelID', 'MessageID']).apply(lambda x: list(zip(x['issuerid'].to_list(), x['SentimentScore'].to_list()))).reset_index()
mes_many_issuers_sentiment  = mes_many.merge(tmp, on = ['ChannelID', 'MessageID'], how ='right')
mes_many_issuers_sentiment = mes_many_issuers_sentiment.dropna()
mes_many_issuers_sentiment = mes_many_issuers_sentiment.rename(columns={0: 'sentiment'})

In [9]:
mes_many_issuers_sentiment

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward,sentiment
0,103,1001029560,2021-02-06 00:52:00,2016-01-14 07:15:26,http://bcs-express.ru/novosti-i-analitika/prog...,False,"[(-1, 3)]"
1,513,1001029560,2021-02-06 00:52:00,2016-06-09 09:47:40,http://bcs-express.ru/novosti-i-analitika/divi...,False,"[(-1, 3)]"
2,6495,1001029560,2021-02-06 00:52:00,2020-09-24 09:46:38,Группа Mail.ru планирует привлечь $600 млн для...,False,"[(-1, 3)]"
3,7869,1001029560,2021-08-11 02:02:52,2021-08-10 15:58:19,Рыночная стоимость Сбербанка превысила $100 мл...,False,"[(-1, 3)]"
4,12838,1001029560,2023-01-23 20:37:25,2023-01-23 15:02:06,​​🎉 Конкурс — Норильский никель после отчетнос...,False,"[(53, 3)]"
...,...,...,...,...,...,...,...
7450,1627,1565800335,2023-06-16 13:23:06,2023-06-16 07:30:00,​🔎 АФК Система (AFKS) - вот кто может оказатьс...,False,"[(204, 3)]"
7451,1632,1565800335,2023-06-19 13:25:14,2023-06-19 07:30:15,​​💎 Алроса (ALRS) - актуализация взгляда на фо...,False,"[(4, 5), (127, 4)]"
7452,1641,1565800335,2023-06-22 13:24:38,2023-06-22 08:32:09,"""​🔥 Новатэк (NVTK) - новые данные с ПМЭФ и про...",False,"[(115, 2)]"
7453,1664,1565800335,2023-07-03 19:32:23,2023-07-03 15:24:56,"⚡ Идеи, которые еще не потеряли актуальность ...",False,"[(12, 4), (4, 4), (111, 4), (53, 4), (58, 4)]"


In [10]:
mes_many_issuers_sentiment['label'] = mes_many_issuers_sentiment['sentiment'].apply(lambda x : x[0][1])

In [11]:
mes_many_issuers_sentiment = mes_many_issuers_sentiment[mes_many_issuers_sentiment['sentiment'].str.len() == 1].copy()

In [12]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
def lemmatize(text):
    words = []
    text = str(text)
 
    """# split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()"""

    doc = Doc(text)
    doc.segment(segmenter)

    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    words += [_.lemma for _ in doc.tokens]

    return words

In [13]:
import re

def remove_words(string, words):
    pattern = r'\b(?:{})\b'.format('|'.join(words))
    
    modified_string = re.sub(pattern, '', string) # уберем стопслова

    modified_string = re.sub(r'[^\w\s]', ' ', modified_string) # уберем пунктуацию
    modified_string = re.sub(r"\b20[0-9]{2}\b", '', modified_string) # Уберем года
    modified_string = re.sub(r"\b[0-9]\b", '', modified_string)

    modified_string = " ".join(modified_string.split())

    
    return modified_string

from nltk.corpus import stopwords
russian_stopwords = stopwords.words('russian') + ['ООО'] + ['ооо']

In [14]:
mes_many_issuers_sentiment['MessageText_lemmatized'] = mes_many_issuers_sentiment['MessageText'].apply(lambda x: " ".join(lemmatize(x))+" ")
mes_many_issuers_sentiment['MessageText_lemmatized'] = mes_many_issuers_sentiment['MessageText_lemmatized'].apply(lambda x: remove_words(x, russian_stopwords))

In [15]:
from nltk.probability import FreqDist
freq_dict = FreqDist(' '.join(mes_many_issuers_sentiment['MessageText_lemmatized'].tolist()).split(' '))

In [16]:
mes_many_issuers_sentiment['name'] = mes_many_issuers_sentiment['sentiment'].apply(lambda x : x[0][0]).astype(int).map(lambda x: id_to_name(x, issuers))

In [17]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(mes_many_issuers_sentiment[['MessageText', 'label', 'name']], test_size=0.2, random_state = 42, stratify=mes_many_issuers_sentiment['label'])

In [19]:
freq_dict

FreqDist({'год': 6773, 'компания': 5499, 'г': 4925, 'акция': 4398, 'дивиденд': 3764, 'млрд': 3634, 'руб': 3334, 'рубль': 2714, 'рост': 2553, 'это': 2507, ...})

In [20]:
bow_vocabulary = [key for key, val in freq_dict.most_common(3000)]


def text_to_bow(text):
    """ Функция, позволяющая превратить входную строку в векторное представление на основании модели мешка слов. """

    sent_vec = np.zeros(len(bow_vocabulary))
    counts = FreqDist(text.split())
    for i, token in enumerate(bow_vocabulary):
        if token in counts:
            sent_vec[i] = counts[token]/len(text.split())
    return dict(zip(bow_vocabulary, sent_vec.tolist()))

In [21]:
df_bow = mes_many_issuers_sentiment.copy()
df_bow = df_bow.apply(lambda x: text_to_bow(x['MessageText_lemmatized']), axis = 1, result_type = 'expand')
df_bow['label'] = mes_many_issuers_sentiment['label']

In [22]:
df_bow_train = df_bow.loc[train_df.index.tolist()]
df_bow_test = df_bow.loc[test_df.index.tolist()]

In [23]:
from catboost import CatBoostClassifier, Pool, CatBoostRegressor, CatBoostRanker

In [24]:
df_bow_train.columns[:9]

Index(['год', 'компания', 'г', 'акция', 'дивиденд', 'млрд', 'руб', 'рубль',
       'рост'],
      dtype='object')

In [31]:
df_bow_train

,год,компания,г,акция,дивиденд,млрд,руб,рубль,рост,это,...,крайний,отвечать,реалия,дешевле,диверсифицировать,майский,основать,ежегодно,бывать,label
4294,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
523,0.034043,0.025532,0.000000,0.008511,0.000000,0.017021,0.000000,0.017021,0.012766,0.021277,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
7216,0.040201,0.015075,0.005025,0.000000,0.000000,0.035176,0.035176,0.000000,0.020101,0.010050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
5417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
5292,0.111111,0.000000,0.000000,0.000000,0.000000,0.074074,0.000000,0.074074,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1941,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2913,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1437,0.000000,0.000000,0.000000,0.051546,0.010309,0.000000,0.000000,0.000000,0.020619,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [28]:
catboost = CatBoostClassifier(iterations=9000, thread_count = -1,
                                            #colsample_bylevel = 0.20,
                                            early_stopping_rounds = 400,
                                            random_seed=42)

In [29]:
train_pool = Pool(df_bow_train.drop('label', axis=1), df_bow_train['label'])
test_pool = Pool(df_bow_test.drop('label', axis=1), df_bow_test['label'])

In [30]:
catboost.fit(train_pool, eval_set=test_pool, metric_period=100, verbose=100, save_snapshot=False,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.04811


0:	learn: 1.5678769	test: 1.5691566	best: 1.5691566 (0)	total: 267ms	remaining: 39m 59s
100:	learn: 0.9610422	test: 1.0061130	best: 1.0061130 (100)	total: 6s	remaining: 8m 48s
200:	learn: 0.8792089	test: 0.9535885	best: 0.9535885 (200)	total: 11.4s	remaining: 8m 19s
300:	learn: 0.8163875	test: 0.9210869	best: 0.9210869 (300)	total: 16.7s	remaining: 8m 2s
400:	learn: 0.7683658	test: 0.8999737	best: 0.8999737 (400)	total: 21.9s	remaining: 7m 50s
500:	learn: 0.7315318	test: 0.8874960	best: 0.8874960 (500)	total: 27.2s	remaining: 7m 40s
600:	learn: 0.7039947	test: 0.8798350	best: 0.8798350 (600)	total: 32.5s	remaining: 7m 34s
700:	learn: 0.6775260	test: 0.8716121	best: 0.8716121 (700)	total: 38s	remaining: 7m 29s
800:	learn: 0.6554391	test: 0.8650805	best: 0.8650805 (800)	total: 43.4s	remaining: 7m 23s
900:	learn: 0.6359769	test: 0.8601156	best: 0.8600432 (897)	total: 48.6s	remaining: 7m 16s
1000:	learn: 0.6190174	test: 0.8562124	best: 0.8562124 (1000)	total: 54.1s	remaining: 7m 12s
1100:	

KeyboardInterrupt: 

In [35]:
catboost.get_feature_importance(prettified = True).head(30)

,Feature Id,Importances
0,год,3.357360
1,компания,2.983321
2,цена,2.155678
3,рубль,1.866068
4,рост,1.859223
5,акция,1.849539
6,млрд,1.596673
7,выплачивать,1.445088
8,вырасти,1.315015
9,дивиденд,1.114135


In [37]:
test_df['bow_pred'] = catboost.predict(df_bow_test.drop('label', axis =1 )).round()
#test_df['bow_score'] = catboost.predict_proba(df_bow_test.drop(df_bow_train.columns[:9], axis =1 ))[:, 1]

In [39]:
metrics_df = test_df.groupby('name').apply(func = lambda x: groubpy_func_class_metrics(x, 'bow_pred')).reset_index().drop('level_1', axis = 1).set_index('name')
metrics_df.to_csv('bow_catboost_metrics.csv')
metrics_df

,count,dist_1,dist_2,dist_3,dist_4,dist_5,accuracy,precision_micro,recall_micro,f1_score_micro
name,,,,,,,,,,
-1,281,5,21,205,36,14,0.672598,0.687246,0.672598,0.680075
252,5,0,0,5,0,0,1.000000,1.000000,1.000000,1.000000
Cian PLC (эмитент депозитарных расписок - The Bank of New York Mellon)\n,3,0,1,0,1,1,0.666667,0.750000,0.666667,0.833333
ETALON GROUP PLC (ЭТАЛОН ГРУП ПИЭЛСИ) (эмитент депозитарных расписок - The Bank of New York Mellon (Бэнк оф Нью Йорк Меллон)),10,0,2,4,4,0,0.400000,0.360000,0.400000,0.472222
Fix Price Group Ltd (Фикс Прайс Груп Лтд) (эмитент депозитарных расписок - The Bank of New York Mellon (Бэнк оф Нью Йорк Меллон)),4,0,1,2,0,1,0.250000,1.000000,0.250000,0.666667
...,...,...,...,...,...,...,...,...,...,...
"публичное акционерное общество ""Группа компаний ""Самолет""",28,0,3,4,15,6,0.714286,0.749412,0.714286,0.749167
"публичное акционерное общество ""Корпорация ВСМПО-АВИСМА""",1,0,0,1,0,0,1.000000,1.000000,1.000000,1.000000
"публичное акционерное общество ""НОВАТЭК""",25,0,3,8,13,1,0.480000,0.473939,0.480000,0.538469


In [77]:
from sklearn.metrics import classification_report
print(classification_report(test_df['label'], test_df['bow_pred'], labels = [i for i in range(1, 6)]))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         9
           2       0.64      0.28      0.39       125
           3       0.69      0.78      0.73       534
           4       0.66      0.70      0.68       430
           5       0.78      0.51      0.62        76

    accuracy                           0.68      1174
   macro avg       0.55      0.46      0.48      1174
weighted avg       0.67      0.68      0.66      1174



Добавим бертовские ембединги

In [76]:
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline
from datasets import Dataset
import torch
import torch.nn as nn

In [70]:
#model_name = 'ai-forever/ruBert-large'
#model_name = 'DeepPavlov/rubert-base-cased'

tokenizer_path = r'../../../1st semestr/rubert_my_v2_tokinizer'
model_path = r'../../../1st semestr/rubert_my_v2'


if 'model_name' in globals() :
    tokenizer_path = model_name
    model_path = model_name

In [71]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [73]:
tokenizer.model_max_length = 512

In [80]:
model = AutoModel.from_pretrained(model_path,
                                                            #attention_probs_dropout_prob=attention_probs_dropout_prob,
                                                            #hidden_dropout_prob=hidden_dropout_prob,
                                                            #classifier_dropout = classifier_dropout
                                                            )

Some weights of BertModel were not initialized from the model checkpoint at ../../../1st semestr/rubert_my_v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
classifier = pipeline("feature-extraction", model=model, tokenizer=tokenizer, device=0, truncation=True, max_length =  512)

In [82]:
res = []

for out in classifier(test_df['MessageText'].to_list()):
    res += [out]

In [102]:
res = [item[0][0] for item in res]
res = np.array(res)

In [114]:
df_combined_test = pd.concat([df_bow_test, pd.DataFrame(res, index=df_bow_test.index, columns =['emb_' + str(i) for i in range(res[0].shape[0])]) ], axis= 1)

In [ ]:
res = []

for out in classifier(train_df['MessageText'].to_list()):
    res += [out]

In [116]:
res = [item[0][0] for item in res]
res = np.array(res)

In [117]:
df_combined_train = pd.concat([df_bow_train, pd.DataFrame(res, index=df_bow_train.index, columns =['emb_' + str(i) for i in range(res[0].shape[0])]) ], axis= 1)

In [171]:
catboost_comb = CatBoostClassifier(iterations=9000, thread_count = -1,
                                            #colsample_bylevel = 0.10,
                                            early_stopping_rounds = 400,
                                            random_seed=42)

In [172]:
train_pool = Pool(df_combined_train.drop(df_bow_train.columns[:9], axis = 1), df_bow_train['label'])
test_pool = Pool(df_combined_test.drop(df_bow_train.columns[:9], axis = 1), df_bow_test['label'])

In [173]:
catboost_comb.fit(train_pool, eval_set=test_pool, metric_period=100, verbose=100, save_snapshot=False,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.04811
0:	learn: 1.5637196	test: 1.5646298	best: 1.5646298 (0)	total: 171ms	remaining: 25m 40s


100:	learn: 0.8797032	test: 0.9646141	best: 0.9646141 (100)	total: 13.8s	remaining: 20m 14s
200:	learn: 0.7502457	test: 0.9151710	best: 0.9151710 (200)	total: 27.3s	remaining: 19m 54s
300:	learn: 0.6453647	test: 0.8856327	best: 0.8856327 (300)	total: 40.6s	remaining: 19m 34s
400:	learn: 0.5693122	test: 0.8716811	best: 0.8716240 (399)	total: 54s	remaining: 19m 16s
500:	learn: 0.5095628	test: 0.8615273	best: 0.8615273 (500)	total: 1m 7s	remaining: 19m
600:	learn: 0.4591797	test: 0.8530396	best: 0.8529418 (599)	total: 1m 20s	remaining: 18m 45s
700:	learn: 0.4199912	test: 0.8462086	best: 0.8462086 (700)	total: 1m 33s	remaining: 18m 31s
800:	learn: 0.3840988	test: 0.8436172	best: 0.8435515 (770)	total: 1m 47s	remaining: 18m 17s
900:	learn: 0.3554285	test: 0.8403806	best: 0.8402704 (899)	total: 2m	remaining: 18m 2s
1000:	learn: 0.3274962	test: 0.8384598	best: 0.8383334 (991)	total: 2m 13s	remaining: 17m 48s
1100:	learn: 0.3057399	test: 0.8362036	best: 0.8360528 (1086)	total: 2m 26s	remaining

In [174]:
catboost_comb.get_feature_importance(prettified = True).head(30)

,Feature Id,Importances
0,emb_758,0.677810
1,emb_679,0.636436
2,emb_46,0.626932
3,emb_505,0.557630
4,emb_736,0.536977
5,emb_580,0.512921
6,emb_444,0.497317
7,emb_201,0.483458
8,emb_627,0.464769
9,emb_157,0.463364


In [175]:
test_df['bow_comb_pred'] = catboost_comb.predict(df_combined_test.drop(df_bow_train.columns[:9], axis =1 ))
test_df['bow_comb_score'] = catboost_comb.predict_proba(df_combined_test.drop(df_bow_train.columns[:9], axis =1 ))[:, 1]

In [176]:
test_df

,MessageText,label,bow_pred,bow_score,bow_comb_pred,bow_comb_score
6756,Взгляд на сектор: Цены на платину и палладий с...,2,3,0.103129,3,0.228896
5613,🛴 Whoosh отчитался за 9 месяцев. Рост налицо ...,4,4,0.014645,4,0.016278
4737,❗️🇷🇺#VTBR #дивиденд 26 мая - ВТБ - сд рассмот...,3,3,0.059425,3,0.024117
6381,📊 ПАО «Распадская» Тикер: RASP Идея: Long ⬆️ С...,5,5,0.008090,5,0.038131
321,Новости компаний — обзоры прессы перед открыти...,3,3,0.001090,3,0.060823
...,...,...,...,...,...,...
2234,"#ЦБРФ #MOEX 🏦 ЦБ РФ и Мосбиржа думают, как изб...",3,3,0.056785,3,0.090603
4264,🛢🇷🇺#NVTK НОВАТЭК изучает увеличение перевалки...,4,4,0.108160,4,0.020738
923,Главное к открытию вторника (30.05): #брифинг ...,3,3,0.003896,3,0.076659
3502,"""????????#GAZP #СП2 #keepcalm Глава совдира ""...",3,3,0.062690,3,0.138799


In [177]:
from sklearn.metrics import classification_report
print(classification_report(test_df['label'], test_df['bow_comb_pred'], labels = [i for i in range(1, 6)]))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         9
           2       0.70      0.18      0.29       125
           3       0.70      0.75      0.72       534
           4       0.61      0.75      0.67       430
           5       0.87      0.43      0.58        76

    accuracy                           0.66      1174
   macro avg       0.58      0.42      0.45      1174
weighted avg       0.67      0.66      0.64      1174

